[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/bdc-business-development-company-python.ipynb)

# Analyze BDC (Business Development Company) SEC Filings with Python -- Free, No API Key

Use **edgartools** to analyze Business Development Company (BDC) filings, portfolio investments, and industry data from the SEC in Python -- completely free, no API key or paid subscription required. BDCs are publicly traded companies that invest in small and mid-sized private businesses.

**What you'll learn:**
- Get the official SEC list of all active BDCs
- Search for BDCs by name or ticker
- Access portfolio investment data (Schedule of Investments)
- Use SEC DERA bulk datasets for cross-BDC analysis

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
from edgar import *
from edgar.bdc import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get the Complete List of BDCs in 1 Line

The SEC maintains an authoritative BDC report listing all Business Development Companies. edgartools gives you direct access:

In [ ]:
# Get the official SEC list of all BDCs
bdcs = get_bdc_list()
print(f"Total active BDCs: {len(bdcs)}")
bdcs

## Search for BDCs by Name or Ticker

Use fuzzy matching to find BDCs:

In [ ]:
# Search by name
results = find_bdc("Ares")
for r in results:
    print(f"  {r.name:45s} CIK: {r.cik}")

In [ ]:
# Search by ticker
results = find_bdc("MAIN")
for r in results:
    print(f"  {r.name:45s} CIK: {r.cik}")

## Check if a Company is a BDC

Quickly verify whether a company is a registered BDC:

In [ ]:
# Check by CIK
print(f"Ares Capital (CIK 1287750): {is_bdc_cik(1287750)}")
print(f"Apple (CIK 320193):         {is_bdc_cik(320193)}")

## Access BDC Filings

BDCs file with the SEC just like other public companies. Get their filings using the standard `Company` API:

In [ ]:
# Get filings for Ares Capital (ARCC) -- the largest BDC
arcc = Company("ARCC")
print(f"{arcc.name} ({arcc.cik})")
print(f"Industry: {arcc.sic} - {arcc.industry}")
print()

# Show recent filings
arcc.get_filings().head(10)

## Get Portfolio Investments

BDCs are required to disclose their portfolio holdings. Access the parsed Schedule of Investments:

In [ ]:
# Get portfolio investments for a BDC
arcc_bdc = bdcs.get_by_ticker("ARCC")
investments = arcc_bdc.portfolio_investments()
print(f"Total portfolio investments: {len(investments)}")
investments

## Analyze Investment Data

Convert portfolio investments to a DataFrame for analysis:

In [ ]:
import pandas as pd

# Get investment data as DataFrame
df = investments.to_dataframe()
print(f"Columns: {list(df.columns)}")
print(f"Shape: {df.shape}")
df.head(10)

## Use SEC DERA Bulk Datasets

The SEC's Division of Economic and Risk Analysis (DERA) publishes bulk BDC data extracts. These contain standardized Schedule of Investments data across all BDCs:

In [ ]:
# See available quarterly datasets
quarters = get_available_quarters()
print(f"Available quarters: {len(quarters)}")
for q in quarters[:5]:
    print(f"  {q}")

In [ ]:
# Fetch the latest available dataset
latest = quarters[0]
dataset = fetch_bdc_dataset(latest[0], latest[1])
print(f"Dataset: Q{latest[1]} {latest[0]}")
print(f"Schedule of Investments rows: {len(dataset.soi)}")
dataset.soi.head(10)

## Cross-BDC Analysis with Bulk Data

Use the DERA dataset to compare portfolio characteristics across BDCs:

In [ ]:
# Top BDCs by total fair value of investments
soi = dataset.soi

if 'fair_value' in soi.columns and 'company_name' in soi.columns:
    top_bdcs = (soi.groupby('company_name')['fair_value']
                .sum()
                .sort_values(ascending=False)
                .head(10))
    print("Top 10 BDCs by Portfolio Fair Value:\n")
    for name, value in top_bdcs.items():
        print(f"  {name:50s} ${value:>15,.0f}")
else:
    print("Available columns:", list(soi.columns))
    soi.head()

## BDC Financial Statements

BDCs also file 10-K and 10-Q reports with full XBRL financial data:

In [ ]:
# Get ARCC's latest 10-K financials
tenk = arcc.get_filings(form="10-K")[0]
financials = tenk.xbrl()

if financials:
    income = financials.statements.income_statement()
    income

## Why EdgarTools?

EdgarTools is free and open-source. Compare accessing BDC data:

**With edgartools (free, no API key):**
```python
from edgar.bdc import get_bdc_list, find_bdc, fetch_bdc_dataset
bdcs = get_bdc_list()                      # All active BDCs
arcc = bdcs.get_by_ticker("ARCC")
investments = arcc.portfolio_investments()  # Parsed holdings
dataset = fetch_bdc_dataset(2024, 3)       # DERA bulk data
```

**Without edgartools:**
```python
# Manually download SEC BDC report PDF
# Parse it yourself to get the list of BDCs
# Download each N-2/10-K filing individually
# Parse XML Schedule of Investments by hand
# Download and process DERA CSV files manually
```

With edgartools, BDC data is a first-class feature -- parsed into Python objects with portfolio-level analytics built in.

## Quick Reference

```python
from edgar import *
from edgar.bdc import *
set_identity("your.name@example.com")

# List all BDCs
bdcs = get_bdc_list()

# Search for a BDC
find_bdc("Ares")              # By name
find_bdc("MAIN")             # By ticker

# Check if a company is a BDC
is_bdc_cik(1287750)          # True for ARCC

# Get portfolio investments
arcc = bdcs.get_by_ticker("ARCC")
investments = arcc.portfolio_investments()
df = investments.to_dataframe()

# DERA bulk datasets
quarters = get_available_quarters()
dataset = fetch_bdc_dataset(2024, 3)
dataset.soi                  # Schedule of Investments

# Standard filing access
Company("ARCC").get_filings(form="10-K")
```

## What's Next

You've learned how to analyze BDC SEC filings with Python. Here are related tutorials:

- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/financial-statements-sec-python.ipynb)
- [Compare Company Financials with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)
- [Analyze Mutual Fund Holdings (N-PORT)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/mutual-fund-holdings-nport-python.ipynb)
- [Search SEC Filings with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/search-sec-filings-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*